In [1]:
#Load third-party packages
import numpy as np
import matplotlib.pyplot as plt
from skimage import transform as tf
from skimage import io

#Load quat, ebsd and hrdic packages from 'defdap' sub-directory
from defdap.quat import Quat
import defdap.ebsd as ebsd
import defdap.hrdic as hrdic


#Set plot behaviour (grain selection has only been tested in osx display mode)
import os
if os.name == 'posix':
    %matplotlib tk
else:
    %matplotlib tk

In [ ]:
def print_maps(self_map, cmap_1=None,vmin_1=0,vmax_1=5): 

        fig= plt.figure(figsize=(10,7))
        plt.imshow(self_map,cmap=cmap_1,vmin=vmin_1,vmax=vmax_1);
        plt.tick_params(axis='both', which='both', bottom='off', top='off', labelbottom='off', right='off', left='off', labelleft='off')
        cbar = plt.colorbar(pad=0.01)
        cbar.set_label(label="$\gamma_{eff}$",size=40)
        cbar.ax.tick_params(labelsize=20)

def calc_mask_val(masked):
    print(np.nanmean(masked))
    print(np.nanpercentile((masked),99))
    print(np.nanpercentile((masked),98))
    print(np.nanpercentile((masked),95))
    print(np.nanpercentile((masked),90))


In [ ]:
# %matplotlib inline

def output(cur_map, vmax1):
    fig= plt.figure(figsize=(15,15))
    plt.imshow(cur_map,cmap='viridis',vmin=0,vmax=vmax1);
    plt.tick_params(axis='both', which='both', bottom='off', top='off', labelbottom='off', right='off', left='off', labelleft='off')
    cbar = plt.colorbar(pad=0.05)
    cbar.set_label(label="$\gamma_{eff}$",size=40)
    cbar.ax.tick_params(labelsize=20)




In [ ]:
plt.colorbar?

In [ ]:
class ab_Maps(object):
    def __init__(self):
        self.f11 = None
        self.f12 = None
        self.f22 = None
        self.Max_shear = None
        self.masks = None
        self.grain_dict = dict()
        
        self.grain_dict['grains'] = dict()
        self.grain_dict['a_grains'] = dict()
        self.grain_dict['b_grains'] = dict()
        self.grain_dict['mixed_grains'] = dict()
        self.grain_dict['ab_stat'] = None

        
    def creat_masks(self, self_map, self_boundaries, xMin, xMax, yMin, yMax, DicMap, bseImage, a_value = 85, b_value = 10):
        x_upper=DicMap.xDim + xMin
        y_upper=DicMap.yDim + yMin
    
        self.masks = dict()
        self.masks['alpha']=np.zeros((y_upper - yMin, x_upper - xMin))
        self.masks['beta']=np.zeros((y_upper - yMin, x_upper - xMin))
        self.masks['mixed']=np.zeros((y_upper - yMin, x_upper - xMin))
        self.masks['small']=np.zeros((y_upper - yMin, x_upper - xMin))
    
        self.masks['BSE_im'] = bseImage[yMin:y_upper,xMin:x_upper]
    
        self.masks['crop_mask']=self_map[yMin:y_upper,xMin:x_upper]
        self.masks['crop_bound']=self_boundaries[yMin:y_upper,xMin:x_upper]

        self.masks['mask_manual_a']=(self.masks['crop_mask'] == 0)
        self.masks['manual_b']=(self.masks['crop_mask'] == 255)


        self.grain_dict['ab_stat'] = ["Fractions [Number, Area]","Alpha", 0, 0,"Beta", 0, 0,"Mixed", 0, 0]
        area_map=np.shape(DicMap.grains)[0]*np.shape(DicMap.grains)[1]


### This loop asign an average greyscale value to each grain, based on the ab_mask
        grains_temp= dict()
        row=0
        for x in DicMap.grains:
            colum=0
            for y in x:
                if y not in grains_temp:
                    grains_temp[y] = [1,0,0]
                    grains_temp[y][1] = self.masks['crop_mask'][row, colum]/255
                else:
                    grains_temp[y][0] += 1
                    grains_temp[y][1] += self.masks['crop_mask'][row, colum]/255
                colum += 1
            row +=1      

        for z in grains_temp:
            u = grains_temp[z][1] / grains_temp [z][0] * 100
            self.grain_dict['grains'][z] = int(u)
            if u > a_value:
                self.grain_dict['ab_stat'][2] += 1
                self.grain_dict['a_grains'][z] = 1
            elif u < b_value:
                self.grain_dict['ab_stat'][5] += 1
                self.grain_dict['b_grains'][z] = 1
            else:
                self.grain_dict['ab_stat'][8] += 1
                self.grain_dict['mixed_grains'][z] = 1
            
        for x in DicMap.grains:
            for y in x:
                if y in self.grain_dict['a_grains']:
                    self.grain_dict['ab_stat'][3] += 1
                if y in self.grain_dict['b_grains']:
                    self.grain_dict['ab_stat'][6] += 1
                if y in self.grain_dict['mixed_grains']:
                    self.grain_dict['ab_stat'][9] += 1
        
        length_g = len(self.grain_dict['grains'])*100
        self.grain_dict['ab_stat'][2] = int(self.grain_dict['ab_stat'][2]/length_g)
        self.grain_dict['ab_stat'][5] = int(self.grain_dict['ab_stat'][5]/length_g)
        self.grain_dict['ab_stat'][8] = int(self.grain_dict['ab_stat'][8]/length_g)
    
        size_map = np.shape(self.masks['alpha'])[0]*np.shape(self.masks['alpha'])[1]/100
        self.grain_dict['ab_stat'][3] = int(self.grain_dict['ab_stat'][3]/size_map)
        self.grain_dict['ab_stat'][6] = int(self.grain_dict['ab_stat'][6]/size_map)
        self.grain_dict['ab_stat'][9] = int(self.grain_dict['ab_stat'][9]/size_map)
    
    
### Calculating average values for grains, 
### needs to be done properly    
        i=1
        for av in DicMap.calcGrainAv(DicMap.crop(DicMap.max_shear)):
            if i in  self.grain_dict['a_grains']:
                self.grain_dict['a_grains'][i] = av
            if i in  self.grain_dict['a_grains']:
                self.grain_dict['b_grains'][i] = av
            if i in  self.grain_dict['mixed_grains']:
                self.grain_dict['mixed_grains'][i] = av
            i += 1

### Creating masks            
        row = 0
        for j in DicMap.grains:
            colum =0
            for i in j:
                if i == -2:
                    self.masks['small'][row, colum] = False
                    self.masks['alpha'][row, colum] = True
                    self.masks['beta'][row, colum] = True
                    self.masks['mixed'][row, colum] = True
                if i in self.grain_dict['a_grains']:
                    self.masks['small'][row, colum] = True
                    self.masks['alpha'][row, colum] = False
                    self.masks['beta'][row, colum] = True
                    self.masks['mixed'][row, colum] = True
                elif i in self.grain_dict['b_grains']:
                    self.masks['small'][row, colum] = True
                    self.masks['alpha'][row, colum] = True
                    self.masks['beta'][row, colum] = False
                    self.masks['mixed'][row, colum] = True

                else:
                    self.masks['small'][row, colum] = True
                    self.masks['alpha'][row, colum] = True
                    self.masks['beta'][row, colum] = True
                    self.masks['mixed'][row, colum] = False
                colum += 1
            row += 1
            
        
    def creat_def_map(self, DicMap):
        self.max_shear = Map_transform(self.masks)
        self.max_shear.transform(DicMap.crop(DicMap.max_shear))
        
        self.f11 = Map_transform(self.masks)
        self.f11.transform(DicMap.crop(DicMap.f11))
        
        self.f12 = Map_transform(self.masks)
        self.f12.transform(DicMap.crop(DicMap.f12))
        
        self.f22 = Map_transform(self.masks)
        self.f22.transform(DicMap.crop(DicMap.f22))
        
        self.BSE = Map_transform(self.masks)
        self.BSE.transform(self.masks['BSE_im'])
        
          
class  Map_transform():
    def __init__(self, imp_dict):
        self.alpha = None
        self.beta = None
        self.mixed = None
        self.dict = imp_dict
        
    def transform(self, def_map):       
        self.alpha = np.ma.array(def_map, mask=self.dict['alpha'])
        self.beta = np.ma.array(def_map, mask=self.dict['beta']) 
        self.mixed = np.ma.array(def_map, mask=self.dict['mixed'])

# Load in DIC and EBSD map

### 05-Macro-1Ks

In [2]:
DicFilePath_5AI = "../Data/05 - MO/Output/"
DicMap_5AI_70 = hrdic.Map(DicFilePath_5AI, "B00001.txt")
DicMap_5AI_80 = hrdic.Map(DicFilePath_5AI, "B00002.txt")
DicMap_5AI_90 = hrdic.Map(DicFilePath_5AI, "B00003.txt")

scale_factor_5AI = 30 / 2048 *24

DicMap_5AI_70.setScale(micrometrePerPixel=scale_factor_5AI)
DicMap_5AI_80.setScale(micrometrePerPixel=scale_factor_5AI)
DicMap_5AI_90.setScale(micrometrePerPixel=scale_factor_5AI)

DicMap_5AI_70.setPatternPath('BSE - 24x24.bmp',1)
DicMap_5AI_80.setPatternPath('BSE - 24x24.bmp',1)
DicMap_5AI_90.setPatternPath('BSE - 24x24.bmp',1)

#DicMap_5AI_70.DicFilePath_5AI = "../Data/05 - MO/Output/"
DicMap_5AI_70 = hrdic.Map(DicFilePath_5AI, "B00001.txt")
DicMap_5AI_80 = hrdic.Map(DicFilePath_5AI, "B00002.txt")
DicMap_5AI_90 = hrdic.Map(DicFilePath_5AI, "B00003.txt")

scale_factor_5AI = 30 / 2048 *24

DicMap_5AI_70.setScale(micrometrePerPixel=scale_factor_5AI)
DicMap_5AI_80.setScale(micrometrePerPixel=scale_factor_5AI)
DicMap_5AI_90.setScale(micrometrePerPixel=scale_factor_5AI)

DicMap_5AI_70.setPatternPath('BSE - 24x24.bmp',1)
DicMap_5AI_80.setPatternPath('BSE - 24x24.bmp',1)
DicMap_5AI_90.setPatternPath('BSE - 24x24.bmp',1)

DicMap_5AI_70.setCrop(xMin=5, xMax=25, yMin=5, yMax=5)
DicMap_5AI_80.setCrop(xMin=5, xMax=25, yMin=5, yMax=5)
DicMap_5AI_90.setCrop(xMin=5, xMax=25, yMin=5, yMax=5)


Loaded DaVis 8.4.0 data (dimensions: 895 x 783 pixels, sub-window size: 24 x 24 pixels)
Loaded DaVis 8.4.0 data (dimensions: 895 x 783 pixels, sub-window size: 24 x 24 pixels)
Loaded DaVis 8.4.0 data (dimensions: 895 x 783 pixels, sub-window size: 24 x 24 pixels)
Loaded DaVis 8.4.0 data (dimensions: 895 x 783 pixels, sub-window size: 24 x 24 pixels)
Loaded DaVis 8.4.0 data (dimensions: 895 x 783 pixels, sub-window size: 24 x 24 pixels)
Loaded DaVis 8.4.0 data (dimensions: 895 x 783 pixels, sub-window size: 24 x 24 pixels)


In [3]:
DicMap_5AI_70.plotMaxShear(vmin=0,vmax=0.05)

In [3]:
#Load in EBSD map and calculate misorientation
EbsdFilePath_5AI = "../Data/05 - MO/Output/EBSD/Sample 5 sub - cp"

Ebsd_5AI = ebsd.Map(EbsdFilePath_5AI, "hexagonal")
Ebsd_5AI.loadSlipSystems(filepath="defdap/slip_systems/hexagonal_withca.txt", cOverA=1.54)
Ebsd_5AI.binData = Ebsd_5AI.binData[::-1]  #rotate the map 180 degrees
Ebsd_5AI.buildQuatArray()
Ebsd_5AI.findBoundaries(boundDef = 10)
Ebsd_5AI.findGrains(minGrainSize=50)
Ebsd_5AI.calcGrainMisOri(calcAxis = True)    #This line takes about 20 minutes to run but is 
                                            #required to display misorientaion
#EbsdMap.calcGrainAvOris()   #If you don't run the misoriatation calculation you need to run this instead.
                            #It takes awhile aswell but not as long as the above
    
Ebsd_5AI.calcAverageGrainSchmidFactors(loadVector=np.array([1, 0, 0])) 

Loaded EBSD data (dimensions: 288 x 214 pixels, step size: 1.5 um)


In [4]:
#Ebsd_5AI.setHomogPoint()
#DicMap_5AI_70.setHomogPoint(display='pattern')

In [8]:
Ebsd_5AI.plotBandContrastMap()

In [4]:
ebsd_homog = np.array (((156, 42), (148, 66), (229, 101), (73, 22), (83, 119), (137, 132), (218, 120)
                      ))



DIC_homog = np.array ((
    (507, 105),(477, 215),(819, 360),(155, 26),(199, 482),(428, 529),(771, 457)
    ))




Ebsd_5AI.homogPoints = ebsd_homog
DicMap_5AI_70.homogPoints = DIC_homog
DicMap_5AI_80.homogPoints = DIC_homog
DicMap_5AI_90.homogPoints = DIC_homog

DicMap_5AI_70.linkEbsdMap(Ebsd_5AI)
DicMap_5AI_80.linkEbsdMap(Ebsd_5AI)
DicMap_5AI_90.linkEbsdMap(Ebsd_5AI)

In [8]:
DicMap_5AI_90.plotMaxShear(plotGBs=True, vmax=0.05)

In [ ]:
Ebsd_5AI.plotBoundaryMap()
plt.scatter(x=Ebsd_5AI.homogPoints[:, 0], y=Ebsd_5AI.homogPoints[:, 1], c='y', s=60)

In [ ]:
ab_map_5AI=io.imread('../Data/05 - MO/Output/BSE - 24x24 - ab.bmp')
#ab_boundary_map_5AI=io.imread('../Data/5AII-macro-oil-24x/Output/ab-grain-selection/5-manual grain selection-48x48-boundaries.bmp')
bseImage_5AI = plt.imread('../Data/05 - MO/Output/BSE - 24x24.bmp')

#ab_boundary_map_5AI = ab_boundary_map_5AI[:,40:-23]
ab_map_5AI = ab_map_5AI[5:-6,5:-30]
bseImage_5AI = bseImage_5AI[5:-6,5:-30]

crop_5AI_max_070 = DicMap_5AI_70.max_shear[5:-5,5:-25]
crop_5AI_max_080 = DicMap_5AI_80.max_shear[5:-5,5:-25]
crop_5AI_max_090 = DicMap_5AI_90.max_shear[5:-5,5:-25]

#print(np.shape(crop_5AI_max_070), np.shape(ab_map_5AI))

In [ ]:
ab_5AI_a_70_max = np.ma.filled(np.ma.array(crop_5AI_max_070, mask=(ab_map_5AI>1)),np.nan)
ab_5AI_b_70_max = np.ma.filled(np.ma.array(crop_5AI_max_070, mask=(ab_map_5AI<1)),np.nan)

ab_5AI_a_80_max = np.ma.filled(np.ma.array(crop_5AI_max_080, mask=(ab_map_5AI>1)),np.nan)
ab_5AI_b_80_max = np.ma.filled(np.ma.array(crop_5AI_max_080, mask=(ab_map_5AI<1)),np.nan)

ab_5AI_a_90_max = np.ma.filled(np.ma.array(crop_5AI_max_090, mask=(ab_map_5AI>1)),np.nan)
ab_5AI_b_90_max = np.ma.filled(np.ma.array(crop_5AI_max_090, mask=(ab_map_5AI<1)),np.nan)

In [ ]:
DicMap_5AI_70.plotMaxShear(plotGBs=True, vmax=0.02)

In [ ]:
#hist = np.histogram(ab_5AI_b_70_max, bins=150, range=(0,0.20), normed=True)


#i=0
#for h in hist[0]:
#    print(hist[0][i])
#    i += 1

In [ ]:
xy=np.invert(ab_map_5AI)
io.imshow(ab_map_5AI)

In [ ]:
#weights = ab_map_5AI

#weights = np.invert(ab_map_5AI)

#a = np.histogram(crop_5AI_max_070, bins=100, range=(0,0.2), weights=weights, normed=True)

#i=0
#for ind in a[0]:
#    print(a[0][i])
#    i += 1

In [ ]:
output(ab_5AI_a_80_max, 0.20)

In [ ]:
the_map = crop_5AI_max_090

print(np.mean(the_map))
print(np.percentile(the_map, 99))
print(np.percentile(the_map, 98))
print(np.percentile(the_map, 95))
print(np.percentile(the_map, 90))

In [ ]:
calc_mask_val(ab_5AI_a_70_max)
calc_mask_val(ab_5AI_b_70_max)
calc_mask_val(ab_5AI_a_80_max)
calc_mask_val(ab_5AI_b_80_max)
calc_mask_val(ab_5AI_a_90_max)
calc_mask_val(ab_5AI_b_90_max)



In [ ]:
#the_map = DicMap_5AI_70.f22

#print(np.mean(the_map))
#print(np.percentile(the_map, 99))
#print(np.percentile(the_map, 98))
#print(np.percentile(the_map, 95))
#print(np.percentile(the_map, 90))

### 06-Macro-1Ks

In [5]:
#Load in DIC map
DicFilePath_6AII = "../Data/06 - M1Ks/Output/"
DicMap_6AII_070 = hrdic.Map(DicFilePath_6AII, "B00001.txt")
DicMap_6AII_080 = hrdic.Map(DicFilePath_6AII, "B00002.txt")
DicMap_6AII_090 = hrdic.Map(DicFilePath_6AII, "B00003.txt")
DicMap_6AII_100 = hrdic.Map(DicFilePath_6AII, "B00004.txt")

scale_factor_6AII = 30 / 2048 *24

DicMap_6AII_070.setScale(micrometrePerPixel=scale_factor_6AII)
DicMap_6AII_080.setScale(micrometrePerPixel=scale_factor_6AII)
DicMap_6AII_090.setScale(micrometrePerPixel=scale_factor_6AII)

DicMap_6AII_070.setPatternPath('BSE - 24x24.bmp',1)
DicMap_6AII_080.setPatternPath('BSE - 24x24.bmp',1)
DicMap_6AII_090.setPatternPath('BSE - 24x24.bmp',1)

DicMap_6AII_070.setCrop(xMin=10, xMax=40, yMin=100, yMax=15)
DicMap_6AII_080.setCrop(xMin=10, xMax=40, yMin=100, yMax=15)
DicMap_6AII_090.setCrop(xMin=10, xMax=40, yMin=100, yMax=15)

Loaded DaVis 8.4.0 data (dimensions: 568 x 556 pixels, sub-window size: 24 x 24 pixels)
Loaded DaVis 8.4.0 data (dimensions: 568 x 556 pixels, sub-window size: 24 x 24 pixels)
Loaded DaVis 8.4.0 data (dimensions: 568 x 556 pixels, sub-window size: 24 x 24 pixels)
Loaded DaVis 8.4.0 data (dimensions: 568 x 556 pixels, sub-window size: 24 x 24 pixels)


In [15]:
DicMap_6AII_090.plotMaxShear(plotGBs=True, vmax=0.05)

In [6]:
#Load in EBSD map and calculate misorientation
EbsdFilePath_6AII = "../Data/06 - M1Ks/Output/EBSD/Ti834_5AI-4-_s1_DIC-cropped"

Ebsd_6AII = ebsd.Map(EbsdFilePath_6AII, "hexagonal")
Ebsd_6AII.loadSlipSystems(filepath="defdap/slip_systems/hexagonal_withca.txt", cOverA=1.54)
#Ebsd_6AII.binData = Ebsd_6AII.binData[::-1]  #rotate the map 180 degrees
Ebsd_6AII.buildQuatArray()
Ebsd_6AII.findBoundaries(boundDef = 10)
Ebsd_6AII.findGrains(minGrainSize=50)
Ebsd_6AII.calcGrainMisOri(calcAxis = True)    #This line takes about 20 minutes to run but is 
                                            #required to display misorientaion
#EbsdMap.calcGrainAvOris()   #If you don't run the misoriatation calculation you need to run this instead.
                            #It takes awhile aswell but not as long as the above
    
Ebsd_6AII.calcAverageGrainSchmidFactors(loadVector=np.array([1, 0, 0])) 

Loading EBSD data...Error with cpr file. Number of phases mismatch.
Loaded EBSD data (dimensions: 649 x 679 pixels, step size: 0.75 um)


In [4]:
#Ebsd_6AII.setHomogPoint()
#DicMap_6AII_070.setHomogPoint(display='pattern')

In [6]:
#DicMap_6AII_070.homogPoints

[(389, 390), (26, 241), (164, 356), (157, 241), (172, 115), (468, 91)]

In [7]:
ebsd_homog = np.array((
    (473, 581), (156, 461), (268, 531), (267, 460), (277, 348), (533, 320)
    ))


DIC_homog = np.array((
    (389, 390), (26, 241), (164, 356), (157, 241), (172, 115), (468, 91)
    ))

Ebsd_6AII.homogPoints = ebsd_homog
DicMap_6AII_070.homogPoints = DIC_homog
DicMap_6AII_080.homogPoints = DIC_homog
DicMap_6AII_090.homogPoints = DIC_homog

DicMap_6AII_070.linkEbsdMap(Ebsd_6AII)
DicMap_6AII_080.linkEbsdMap(Ebsd_6AII)
DicMap_6AII_090.linkEbsdMap(Ebsd_6AII)

In [9]:
DicMap_6AII_090.plotMaxShear(plotGBs=True, vmax=0.02)

In [ ]:
ab_map_6AII=io.imread('../Data/06 - M1Ks/Output/BSE - 24x24 - ab.bmp')
#ab_boundary_map_6AII=io.imread('../Data/6AII-1Ks-16x/Output/ab-grain-selection/macro_1ks_ab_boundary.bmp')
bseImage_6AII = plt.imread('../Data/06 - M1Ks/Output/BSE - 24x24 - ab.bmp')

ab_map_6AII = ab_map_6AII[100:-21,10:-46]
#ab_boundary_map_6AII = ab_boundary_map_6AII[200:-27,190:-61]
bseImage_6AII = bseImage_6AII[100:-21,10:-46]

crop_6AII_max_070 = DicMap_6AII_070.max_shear[100:-20,10:-40]
crop_6AII_max_080 = DicMap_6AII_080.max_shear[100:-20,10:-40]
crop_6AII_max_090 = DicMap_6AII_090.max_shear[100:-20,10:-40]

#print(np.shape(ab_map_6AII), np.shape(crop_6AII_max_070))

In [ ]:
ab_6AII_a_70_max = np.ma.filled(np.ma.array(crop_6AII_max_070, mask=(ab_map_6AII>1)),np.nan)
ab_6AII_b_70_max = np.ma.filled(np.ma.array(crop_6AII_max_070, mask=(ab_map_6AII<1)),np.nan)

ab_6AII_a_80_max = np.ma.filled(np.ma.array(crop_6AII_max_080, mask=(ab_map_6AII>1)),np.nan)
ab_6AII_b_80_max = np.ma.filled(np.ma.array(crop_6AII_max_080, mask=(ab_map_6AII<1)),np.nan)

ab_6AII_a_90_max = np.ma.filled(np.ma.array(crop_6AII_max_090, mask=(ab_map_6AII>1)),np.nan)
ab_6AII_b_90_max = np.ma.filled(np.ma.array(crop_6AII_max_090, mask=(ab_map_6AII<1)),np.nan)

In [ ]:
hist = np.histogram(ab_6AII_b_90_max, bins=100, range=(0,0.1), normed=True)


i=0
for h in hist[0]:
    print(hist[0][i])
    i += 1

In [ ]:
output(ab_6AII_b_90_max, 0.20)

In [ ]:
#calc_mask_val(ab_6AII_a_70_max)
#calc_mask_val(ab_6AII_b_70_max)
#calc_mask_val(ab_6AII_a_80_max)
#calc_mask_val(ab_6AII_b_80_max)
#calc_mask_val(ab_6AII_a_90_max)
#calc_mask_val(ab_6AII_b_90_max)

the_map = crop_6AII_max_090

print(np.mean(the_map))
print(np.percentile(the_map, 99))
print(np.percentile(the_map, 98))
print(np.percentile(the_map, 95))
print(np.percentile(the_map, 90))

In [ ]:
io.imshow(ab_map_6AII)

In [ ]:

ab_6AII_a_70_max = np.ma.array(crop_6AII_max_070, mask=(ab_map_6AII<1))


In [ ]:
fig= plt.figure(figsize=(10,10))
plt.imshow(ab_6AII_a_70_max,cmap='viridis',vmin=0,vmax=0.05);
plt.tick_params(axis='both', which='both', bottom='off', top='off', labelbottom='off', right='off', left='off', labelleft='off')
cbar = plt.colorbar(pad=0.01)
cbar.set_label(label="$\gamma_{eff}$",size=40)
cbar.ax.tick_params(labelsize=20)

In [ ]:
the_map = crop_6AII_max_090

print(np.mean(the_map))
print(np.percentile(the_map, 99))
print(np.percentile(the_map, 98))
print(np.percentile(the_map, 95))
print(np.percentile(the_map, 90))

In [ ]:
io.imshow(ab_5AI_a_70_max)


In [ ]:
DicMap_6AII_070.plotMaxShear()

### 11-NO-1Ks

In [9]:
#Load in DIC map
DicFilePath_6AII_bin = "../Data/11 - N1Ks/Output/"
DicMap_11_70 = hrdic.Map(DicFilePath_6AII_bin, "B00001.txt")
DicMap_11_80 = hrdic.Map(DicFilePath_6AII_bin, "B00002.txt")
DicMap_11_90 = hrdic.Map(DicFilePath_6AII_bin, "B00003.txt")

scale_factor_11 = 30 / 2048 * 24

DicMap_11_70.setScale(micrometrePerPixel=scale_factor_11)
DicMap_11_80.setScale(micrometrePerPixel=scale_factor_11)
DicMap_11_90.setScale(micrometrePerPixel=scale_factor_11)

DicMap_11_70.setPatternPath('BSE - 24x24.bmp',1)
DicMap_11_80.setPatternPath('BSE - 24x24.bmp',1)
DicMap_11_90.setPatternPath('BSE - 24x24.bmp',1)

DicMap_11_70.setCrop(xMin=75, xMax=10, yMin=5, yMax=10)
DicMap_11_80.setCrop(xMin=75, xMax=10, yMin=5, yMax=10)
DicMap_11_90.setCrop(xMin=75, xMax=10, yMin=5, yMax=10)

Loaded DaVis 8.4.0 data (dimensions: 887 x 782 pixels, sub-window size: 24 x 24 pixels)
Loaded DaVis 8.4.0 data (dimensions: 887 x 782 pixels, sub-window size: 24 x 24 pixels)
Loaded DaVis 8.4.0 data (dimensions: 887 x 782 pixels, sub-window size: 24 x 24 pixels)


In [7]:
DicMap_11_90.plotMaxShear(vmax=0.03)

In [2]:
#Load in EBSD map and calculate misorientation
EbsdFilePath_11 = "../Data/11 - N1Ks/Output/EBSD/in-situ-11-ebsd 075um - cleaned"

Ebsd_11_bin = ebsd.Map(EbsdFilePath_11, "hexagonal")
Ebsd_11_bin.loadSlipSystems(filepath="defdap/slip_systems/hexagonal_withca.txt", cOverA=1.54)
#Ebsd_11.binData = Ebsd_11.binData[::-1]  #rotate the map 180 degrees
Ebsd_11_bin.buildQuatArray()
Ebsd_11_bin.findBoundaries(boundDef = 10)
Ebsd_11_bin.findGrains(minGrainSize=50)
Ebsd_11_bin.calcGrainMisOri(calcAxis = True)    #This line takes about 20 minutes to run but is 
                                            #required to display misorientaion
#EbsdMap.calcGrainAvOris()   #If you don't run the misoriatation calculation you need to run this instead.
                            #It takes awhile aswell but not as long as the above
    
Ebsd_11_bin.calcAverageGrainSchmidFactors(loadVector=np.array([1, 0, 0]))   

Loading EBSD data...Error with cpr file. Number of phases mismatch.
Loaded EBSD data (dimensions: 604 x 454 pixels, step size: 0.75 um)


In [11]:
Ebsd_11_bin.setHomogPoint()
DicMap_11_90.setHomogPoint(display='pattern')


In [15]:
Ebsd_11_bin.homogPoints

[(409, 255),
 (378, 286),
 (264, 276),
 (243, 150),
 (428, 56),
 (481, 145),
 (315, 162),
 (347, 129),
 (420, 226)]

In [17]:
ebsd_homog = np.array((
    (409, 255),(378, 286),(264, 276),(243, 150),(428, 56),(481, 145),(315, 162),(347, 129),(420, 226)
    ))


DIC_homog = np.array((
    (525, 490),(448, 549),(201, 498),(182, 218),(609, 52),(708, 264),(337, 261),(414, 195),(555, 431)
    ))

Ebsd_11_bin.homogPoints = ebsd_homog
DicMap_11_70.homogPoints = DIC_homog
DicMap_11_80.homogPoints = DIC_homog
DicMap_11_90.homogPoints = DIC_homog

DicMap_11_70.linkEbsdMap(Ebsd_11_bin)
DicMap_11_80.linkEbsdMap(Ebsd_11_bin)
DicMap_11_90.linkEbsdMap(Ebsd_11_bin)

In [19]:
DicMap_11_80.plotMaxShear(plotGBs=True, vmax=0.05)

In [ ]:
ab_map_11=io.imread('../Data/11 - N1Ks/Output/BSE - 24x24 - ab.bmp')
#ab_boundary_map_6AII=io.imread('../Data/6AII-1Ks-16x/Output/ab-grain-selection/macro_1ks_ab_boundary.bmp')
bseImage_11 = plt.imread('../Data/11 - N1Ks/Output/BSE - 24x24.bmp')

ab_map_11 = ab_map_11[5:-11,75:-15]
#ab_boundary_map_6AII = ab_boundary_map_6AII[200:-27,190:-61]
bseImage_11 = bseImage_11[5:-11,75:-15]

crop_11_max_070 = DicMap_11_70.max_shear[5:-10,75:-10]
crop_11_max_080 = DicMap_11_80.max_shear[5:-10,75:-10]
crop_11_max_090 = DicMap_11_90.max_shear[5:-10,75:-10]

print(np.shape(ab_map_11), np.shape(crop_11_max_070))


In [ ]:
ab_11_a_70_max = np.ma.filled(np.ma.array(crop_11_max_070, mask=(ab_map_11>1)),np.nan)
ab_11_b_70_max = np.ma.filled(np.ma.array(crop_11_max_070, mask=(ab_map_11<1)),np.nan)

ab_11_a_80_max = np.ma.filled(np.ma.array(crop_11_max_080, mask=(ab_map_11>1)),np.nan)
ab_11_b_80_max = np.ma.filled(np.ma.array(crop_11_max_080, mask=(ab_map_11<1)),np.nan)

ab_11_a_90_max = np.ma.filled(np.ma.array(crop_11_max_090, mask=(ab_map_11>1)),np.nan)
ab_11_b_90_max = np.ma.filled(np.ma.array(crop_11_max_090, mask=(ab_map_11<1)),np.nan)

In [ ]:
hist = np.histogram(ab_11_b_90_max, bins=150, range=(0,0.20), normed=True)


i=0
for h in hist[0]:
    print(hist[0][i])
    i += 1

In [ ]:
#output(ab_11_b_90_max, 0.20)

In [ ]:
#the_map = crop_11_max_090

#print(np.mean(the_map))
#print(np.percentile(the_map, 99))
#print(np.percentile(the_map, 98))
#print(np.percentile(the_map, 95))
#print(np.percentile(the_map, 90))

#calc_mask_val(ab_11_a_70_max)
#calc_mask_val(ab_11_b_70_max)
#calc_mask_val(ab_11_a_80_max)
#calc_mask_val(ab_11_b_80_max)
#calc_mask_val(ab_11_a_90_max)
#calc_mask_val(ab_11_b_90_max)

### 11-No-macro-1Ks   with unloading

In [ ]:
#Load in DIC map
DicFilePath_11A = "../Data/N1-11-repeated/Output/"
DicMap_11A_70 = hrdic.Map(DicFilePath_11A, "N1-11-48x48 (1).txt")
DicMap_11A_70_un = hrdic.Map(DicFilePath_11A, "N1-11-48x48 (2).txt")
DicMap_11A_80 = hrdic.Map(DicFilePath_11A, "N1-11-48x48 (3).txt")
DicMap_11A_80_un = hrdic.Map(DicFilePath_11A, "N1-11-48x48 (4).txt")
DicMap_11A_90 = hrdic.Map(DicFilePath_11A, "N1-11-48x48 (5).txt")

scale_factor_6AII = 30 / 2048 *48

DicMap_11A_70.setScale(micrometrePerPixel=scale_factor_6AII)
DicMap_11A_70_un.setScale(micrometrePerPixel=scale_factor_6AII)
DicMap_11A_80.setScale(micrometrePerPixel=scale_factor_6AII)
DicMap_11A_80_un.setScale(micrometrePerPixel=scale_factor_6AII)
DicMap_11A_90.setScale(micrometrePerPixel=scale_factor_6AII)

DicMap_11A_70.setCrop(xMin=0, xMax=20, yMin=10, yMax=80)
DicMap_11A_70_un.setCrop(xMin=0, xMax=20, yMin=10, yMax=80)
DicMap_11A_80.setCrop(xMin=0, xMax=20, yMin=10, yMax=80)
DicMap_11A_80_un.setCrop(xMin=0, xMax=20, yMin=10, yMax=80)
DicMap_11A_90.setCrop(xMin=0, xMax=20, yMin=10, yMax=80)

DicMap_11A_70.setPatternPath('BSE-images/B00001',4)
DicMap_11A_70_un.setPatternPath('BSE-images/B00001',4)
DicMap_11A_80.setPatternPath('BSE-images/B00001',4)
DicMap_11A_80_un.setPatternPath('BSE-images/B00001',4)
DicMap_11A_90.setPatternPath('BSE-images/B00001',4)

In [ ]:
DicMap_11A_80_un.plotMaxShear(vmax=0.1)

In [ ]:
DicMap_11A_70.plotMaxShear()

### 12-No-No-macro-oil

In [19]:
DicFilePath_12 =  "../Data/12 - NO/Output/"
DicMap_12_70 = hrdic.Map(DicFilePath_12, "B00001.txt")
DicMap_12_80 = hrdic.Map(DicFilePath_12, "B00002.txt")
DicMap_12_90 = hrdic.Map(DicFilePath_12, "B00003.txt")

scale_factor_12 = 30 / 2048 * 24

DicMap_12_70.setScale(micrometrePerPixel=scale_factor_12)
DicMap_12_80.setScale(micrometrePerPixel=scale_factor_12)
DicMap_12_90.setScale(micrometrePerPixel=scale_factor_12)

DicMap_12_70.setPatternPath('BSE - 24x24.bmp',1)
DicMap_12_80.setPatternPath('BSE - 24x24.bmp',1)
DicMap_12_90.setPatternPath('BSE - 24x24.bmp',1)

DicMap_12_70.setCrop(xMin=13, xMax=80, yMin=62, yMax=5)
DicMap_12_80.setCrop(xMin=13, xMax=80, yMin=62, yMax=5)
DicMap_12_90.setCrop(xMin=13, xMax=80, yMin=62, yMax=5)

Loaded DaVis 8.4.0 data (dimensions: 840 x 737 pixels, sub-window size: 24 x 24 pixels)
Loaded DaVis 8.4.0 data (dimensions: 840 x 737 pixels, sub-window size: 24 x 24 pixels)
Loaded DaVis 8.4.0 data (dimensions: 840 x 737 pixels, sub-window size: 24 x 24 pixels)


In [20]:
#Load in EBSD map and calculate misorientation
EbsdFilePath_12 = "../Data/12 - NO/Output/EBSD/12-no-macro-oil"

Ebsd_12_bin = ebsd.Map(EbsdFilePath_12, "hexagonal")
Ebsd_12_bin.loadSlipSystems(filepath="defdap/slip_systems/hexagonal_withca.txt", cOverA=1.54)
#Ebsd_6AII.binData = Ebsd_6AII.binData[::-1]  #rotate the map 180 degrees
Ebsd_12_bin.buildQuatArray()
Ebsd_12_bin.findBoundaries(boundDef = 10)
Ebsd_12_bin.findGrains(minGrainSize=50)
Ebsd_12_bin.calcGrainMisOri(calcAxis = True)    #This line takes about 20 minutes to run but is 
                                            #required to display misorientaion
#EbsdMap.calcGrainAvOris()   #If you don't run the misoriatation calculation you need to run this instead.
                            #It takes awhile aswell but not as long as the above
    
Ebsd_12_bin.calcAverageGrainSchmidFactors(loadVector=np.array([1, 0, 0]))   

Loading EBSD data...Error with cpr file. Number of phases mismatch.
Loaded EBSD data (dimensions: 500 x 400 pixels, step size: 2.9945 um)


In [21]:
Ebsd_12_bin.setHomogPoint()
DicMap_12_70.setHomogPoint(display='pattern')

In [23]:
Ebsd_12_bin.homogPoints

[(110, 290), (203, 242), (189, 186), (91, 316), (63, 332), (51, 283)]

In [26]:
ebsd_homog = np.array((
    (110, 290), (203, 242), (189, 186), (91, 316), (63, 332), (51, 283)
    ))
(110, 290), (203, 242), (189, 186), (91, 316), (63, 332), (51, 283)

DIC_homog = np.array((
    (260, 413), (534, 258), (490, 64), (204, 499), (119, 547), (89, 390)
    ))

Ebsd_12_bin.homogPoints = ebsd_homog
DicMap_12_70.homogPoints = DIC_homog
DicMap_12_80.homogPoints = DIC_homog
DicMap_12_90.homogPoints = DIC_homog

DicMap_12_70.linkEbsdMap(Ebsd_12_bin)
DicMap_12_80.linkEbsdMap(Ebsd_12_bin)
DicMap_12_90.linkEbsdMap(Ebsd_12_bin)

In [31]:
DicMap_12_90.plotMaxShear(plotGBs=True, vmax=0.05)

In [ ]:
ab_map_12=io.imread('../Data/12 - NO/Output/BSE - 24x24 - ab.bmp')
#ab_boundary_map_6AII=io.imread('../Data/6AII-1Ks-16x/Output/ab-grain-selection/macro_1ks_ab_boundary.bmp')
bseImage_12 = plt.imread('../Data/12 - NO/Output/BSE - 24x24.bmp')

ab_map_12 = ab_map_12[62:-6,13:-86]
#ab_boundary_map_6AII = ab_boundary_map_6AII[200:-27,190:-61]
bseImage_12 = bseImage_12[62:-6,13:-86]

crop_12_max_70 = DicMap_12_70.max_shear[62:-5,13:-80]
crop_12_max_80 = DicMap_12_80.max_shear[62:-5,13:-80]
crop_12_max_90 = DicMap_12_90.max_shear[62:-5,13:-80]

print(np.shape(ab_map_12), np.shape(crop_12_max_70))

In [ ]:
ab_12_a_70_max = np.ma.filled(np.ma.array(crop_12_max_70, mask=(ab_map_12>1)),np.nan)
ab_12_b_70_max = np.ma.filled(np.ma.array(crop_12_max_70, mask=(ab_map_12<1)),np.nan)

ab_12_a_80_max = np.ma.filled(np.ma.array(crop_12_max_80, mask=(ab_map_12>1)),np.nan)
ab_12_b_80_max = np.ma.filled(np.ma.array(crop_12_max_80, mask=(ab_map_12<1)),np.nan)

ab_12_a_90_max = np.ma.filled(np.ma.array(crop_12_max_90, mask=(ab_map_12>1)),np.nan)
ab_12_b_90_max = np.ma.filled(np.ma.array(crop_12_max_90, mask=(ab_map_12<1)),np.nan)

In [ ]:
hist = np.histogram(ab_12_b_90_max, bins=150, range=(0,0.15), normed=True)


i=0
for h in hist[0]:
    print(hist[0][i])
    i += 1

In [ ]:
#the_map = crop_12_max_70

#print(np.mean(the_map))
#print(np.percentile(the_map, 99))
#print(np.percentile(the_map, 98))
#print(np.percentile(the_map, 95))
#print(np.percentile(the_map, 90))

calc_mask_val(ab_12_a_70_max)
calc_mask_val(ab_12_b_70_max)
calc_mask_val(ab_12_a_80_max)
calc_mask_val(ab_12_b_80_max)
calc_mask_val(ab_12_a_90_max)
calc_mask_val(ab_12_b_90_max)

In [ ]:
output(ab_12_a_90_max, 0.20)

In [ ]:
np.histogram

### Load in EBSD map, detect grains thena calculate grain misorientation

In [ ]:
#Load in EBSD map and calculate misorientation
EbsdFilePath_5AI = "../Data/MO-5-repeated-unloading/Output/EBSD/Sample 5 cleaned"

Ebsd_5AI = ebsd.Map(EbsdFilePath_5AI, "hexagonal")
Ebsd_5AI.loadSlipSystems(filepath="defdap/slip_systems/hexagonal_withca.txt", cOverA=1.54)
Ebsd_5AI.binData = Ebsd_5AI.binData[::-1]  #rotate the map 180 degrees
Ebsd_5AI.buildQuatArray()
Ebsd_5AI.findBoundaries(boundDef = 10)
Ebsd_5AI.findGrains(minGrainSize=50)
Ebsd_5AI.calcGrainMisOri(calcAxis = True)    #This line takes about 20 minutes to run but is 
                                            #required to display misorientaion
#EbsdMap.calcGrainAvOris()   #If you don't run the misoriatation calculation you need to run this instead.
                            #It takes awhile aswell but not as long as the above
    
Ebsd_5AI.calcAverageGrainSchmidFactors(loadVector=np.array([1, 0, 0])) 

In [ ]:
#Load in EBSD map and calculate misorientation
EbsdFilePath_11A = "../Data/N1-11-repeated/Output/EBSD/Sample11 Zoom"

Ebsd_11A = ebsd.Map(EbsdFilePath_11A, "hexagonal")
Ebsd_11A.loadSlipSystems(filepath="defdap/slip_systems/hexagonal_withca.txt", cOverA=1.54)
#Ebsd_11A.binData = EbsdFilePath_11A.binData[::-1]  #rotate the map 180 degrees
Ebsd_11A.buildQuatArray()
Ebsd_11A.findBoundaries(boundDef = 10)
Ebsd_11A.findGrains(minGrainSize=50)
Ebsd_11A.calcGrainMisOri(calcAxis = True)    #This line takes about 20 minutes to run but is 
                                            #required to display misorientaion
#EbsdMap.calcGrainAvOris()   #If you don't run the misoriatation calculation you need to run this instead.
                            #It takes awhile aswell but not as long as the above
    
Ebsd_11A.calcAverageGrainSchmidFactors(loadVector=np.array([1, 0, 0])) 

### Loading Pattern 

In [ ]:
#DicMap_5AI_70.setHomogPoint(display='pattern')


## Run these cells when notebook was restarted


In [ ]:
ebsd_homog = np.array((
(291, 351),
 (94, 329),
 (94, 121),
 (289, 276),
 (329, 215),
 (171, 186),
 (358, 78),
 (146, 75),
 (170, 249)
    ))


DIC_homog = np.array((
(416, 536),
 (71, 488),
 (82, 126),
 (416, 405),
 (488, 306),
 (216, 241),
 (552, 63),
 (179, 46),
 (208, 351)
))

Ebsd_6AII.homogPoints = ebsd_homog
DicMap_6AII_70.homogPoints = DIC_homog
DicMap_6AII_80.homogPoints = DIC_homog
DicMap_6AII_90.homogPoints = DIC_homog

DicMap_6AII_70.linkEbsdMap(Ebsd_6AII)
DicMap_6AII_80.linkEbsdMap(Ebsd_6AII)
DicMap_6AII_90.linkEbsdMap(Ebsd_6AII)

In [ ]:
ebsd_homog = np.array((
 (226, 116),
 (237, 93),
 (178, 65),
 (92, 47),
 (173, 87),
 (122, 88),
 (180, 139),
 (144, 156),
 (104, 142),
 (107, 183),
 (224, 146)
))

DIC_homog = np.array ((
 (361, 167),
 (381, 112),
 (256, 53),
 (76, 21),
 (247, 108),
 (139, 112),
 (261, 229),
 (183, 276),
 (100, 242),
 (102, 341),
 (355, 240)
))

Ebsd_5AI.homogPoints = ebsd_homog
DicMap_5AI_70.homogPoints = DIC_homog
DicMap_5AI_80.homogPoints = DIC_homog
DicMap_5AI_90.homogPoints = DIC_homog

DicMap_5AI_70.linkEbsdMap(Ebsd_5AI)
DicMap_5AI_80.linkEbsdMap(Ebsd_5AI)
DicMap_5AI_90.linkEbsdMap(Ebsd_5AI)

### Plot max shear with boundaries

In [ ]:
DicMap_6AII_70.plotMaxShear(vmax=0.05, plotGBs=True)
DicMap_6AII_80.plotMaxShear(vmax=0.05, plotGBs=True)
DicMap_6AII_90.plotMaxShear(vmax=0.05, plotGBs=True)


In [ ]:
DicMap_5AI_70.plotMaxShear(vmax=0.05, plotGBs=True)
DicMap_5AI_80.plotMaxShear(vmax=0.05, plotGBs=True)
DicMap_5AI_90.plotMaxShear(vmax=0.05, plotGBs=True)


In [ ]:
DicMap_5AI_70.setHomogPoint()
Ebsd_5AI.setHomogPoint()

# A-B seperation

In [ ]:
DicMap_6AII_70.findGrains(minGrainSize=100)
#DicMap_6AII_80.findGrains(minGrainSize=100)
#DicMap_6AII_90.findGrains(minGrainSize=100)


In [ ]:
print(emptymap.shape)

In [ ]:
print(range(17))

In [ ]:
selMap = DicMap_6AII_70

ids=range(100)

def plotGrains(ids):

    fig,ax = plt.subplots()
    for id in ids:
        selMap[id].plotMaxShear(ax=ax, refAxes='Map', vmin=0, vmax=8)

plotGrains(ids)

In [ ]:
selMap[16].plotMaxShear()

In [ ]:
ab_map_6AII=io.imread('../Data/6AII-1Ks-16x/Output/ab-grain-selection/macro_1ks_ab.tif')
ab_boundary_map_6AII=io.imread('../Data/6AII-1Ks-16x/Output/ab-grain-selection/macro_1ks_ab_boundary.bmp')
bseImage_6AII = plt.imread('../Data/6AII-1Ks-16x/Output/ab-grain-selection/Macro-1KS-090-Shiftcorrection.bmp')


In [ ]:
Sample_6AII_70=ab_Maps()
Sample_6AII_70.creat_masks(ab_map_6AII, ab_boundary_map_6AII, 190, 50, 200, 30, DicMap_6AII_70, bseImage_6AII)
Sample_6AII_70.creat_def_map(DicMap_6AII_70)

Sample_6AII_80=ab_Maps()
Sample_6AII_80.creat_masks(ab_map_6AII, ab_boundary_map_6AII, 190, 50, 200, 30, DicMap_6AII_80, bseImage_6AII)
Sample_6AII_80.creat_def_map(DicMap_6AII_80)

Sample_6AII_90=ab_Maps()
Sample_6AII_90.creat_masks(ab_map_6AII, ab_boundary_map_6AII, 190, 50, 200, 30, DicMap_6AII_90, bseImage_6AII)
Sample_6AII_90.creat_def_map(DicMap_6AII_90)

In [ ]:
print_maps(Sample_6AII_70.max_shear.alpha, vmin_1=0,vmax_1=0.05)
print_maps(Sample_6AII_80.max_shear.beta, vmin_1=0,vmax_1=0.05)
print_maps(Sample_6AII_90.max_shear.mixed, vmin_1=0,vmax_1=0.05)

In [ ]:
print_maps(Sample_6AII_70.BSE.alpha, vmin_1=0,vmax_1=255)
print_maps(Sample_6AII_70.BSE.beta, vmin_1=0,vmax_1=255)
print_maps(Sample_6AII_70.BSE.mixed, vmin_1=0,vmax_1=255)

In [ ]:
Sample_6AII_70.f11.mixed

In [ ]:
fig= plt.figure(figsize=(10,7))
plt.imshow(Sample1.BSE.mixed, cmap='Greys');
plt.tick_params(axis='both', which='both', bottom='off', top='off', labelbottom='off', right='off', left='off', labelleft='off')
cbar.set_label(label="$\gamma_{eff}$",size=40)
cbar.ax.tick_params(labelsize=20)

## Size and shape

In [ ]:
print("\t\tDIC script\tOld script")
print("x-dimension: \t" + str(DicMap.xDim) + "\t\t" + str(np.shape(cropped_boundary)[1]))
print("y-dimention: \t" + str(DicMap.yDim) + "\t\t" + str(np.shape(cropped_boundary)[0]))

In [ ]:
fraction=0
colum=0
row=0
for x in cropped_mask:
    colum=0
    for y in x:
        fraction += cropped_mask[row, colum]
        colum += 1
    row +=1
fraction=fraction/(np.shape(cropped_mask)[0]*np.shape(cropped_mask)[1])/255
print(fraction)

In [ ]:
DicMap.findGrains()

In [ ]:
DicMap.boundaries[1][10]

In [ ]:
ebsd.Map.


In [ ]:
DicMap.plotMaxShear(vmax=5, plotGBs=True)

# Grain statistics

In [ ]:
DicMap.findGrains(minGrainSize=50)
myGrainMap=DicMap.grains
myNumberGrainsDic=np.max(DicMap.grains)
myShearMap=DicMap.max_shear[200:-30,190:-50]          #enter crop
mySizeConversion=pow((30*16/2048),2)

myGrainListDic=np.zeros(shape=[myNumberGrainsDic,7])
myArray={}

position=0
for x in myGrainListDic:
    myGrainListDic[position,0]=position+1
    position=position+1
    myArray["GrainSize"+str(position)]=0
    myArray["GrainStrain"+str(position)]=0

    
    
myArray["boundaryPixles"]=0
xi=0
for x in myShearMap:
    yi=0 
    for y in x:
        value=myGrainMap[xi,yi]
        if value>0:
            myArray["GrainSize"+str(value)]=myArray["GrainSize"+str(value)]+1
            myArray["GrainStrain"+str(value)]=myArray["GrainStrain"+str(value)]+y
        else:
            myArray["boundaryPixles"]=myArray["boundaryPixles"]+1
        yi=yi+1
    xi=xi+1

position=0
for x in myGrainListDic:
    currentGrain=DicMap.grainList[position]
    SchmidFacCurrent=currentGrain.ebsdGrain.averageSchmidFactors
    prism=(SchmidFacCurrent[1],SchmidFacCurrent[2],SchmidFacCurrent[3])
    pyramidal=(SchmidFacCurrent[4],SchmidFacCurrent[5])
    py_ca=(SchmidFacCurrent[6],SchmidFacCurrent[7],SchmidFacCurrent[8],SchmidFacCurrent[9])
    
    myGrainListDic[position,1]=myArray["GrainSize"+str(position+1)]*mySizeConversion
    myGrainListDic[position,2]=myArray["GrainStrain"+str(position+1)]/myArray["GrainSize"+str(position+1)]*100
    myGrainListDic[position,3]=np.max(SchmidFacCurrent[0])               #Basal
    myGrainListDic[position,4]=np.max(prism)                             #Prism
    myGrainListDic[position,5]=np.max(pyramidal)                         #Pyramidal
    myGrainListDic[position,6]=np.max(py_ca)                             #Pyramidal_c+a
    
    position=position+1

    
    
myGrainListDic_sorted=np.copy(myGrainListDic)
myGrainListDic_size=np.copy(myGrainListDic)
bubble_sort(myGrainListDic_sorted)
bubble_size(myGrainListDic_size)
    
print(myGrainListDic)
print("boundaryPixles", myArray["boundaryPixles"])


### Update sample name and path

In [ ]:
load="070"
sample="Macro-1Ks"

sample_sorted=sample+load+"%-GrainStrain_sorted.txt"
sample_unsorted=sample+load+"%-GrainStrain_unsorted.txt"
path="../Data/6AII-1Ks-16x/Results/"

name="macro-1ks-"+load+"-Gb.png"
DicMap.plotMaxShear(vmax=5, plotGBs=True, plotColourBar=True)
plt.savefig(path+name)

name="macro-1ks-"+load+"-noGb.png"
DicMap.plotMaxShear(vmax=5, plotGBs=False, plotColourBar=True)
plt.savefig(path+name)

f=open(path+sample_unsorted, 'wb')
np.savetxt(f,myGrainListDic,delimiter="\t", fmt="%s")
f.close()

f=open(path+sample_sorted, 'wb')
np.savetxt(f,myGrainListDic_sorted,delimiter="\t", fmt="%s")
f.close()

### Grains with high strain

In [ ]:
myGrains_copy=np.copy(myGrainListDic_sorted)
grainSorted=myGrains_copy[-10:,0]
grainSorted=list(map(int, grainSorted))
print(grainSorted)

In [ ]:
#### Copy array manualy and select

In [ ]:
grainCopy=(102, 94, 75, 119, 66, 149, 92, 195)
folder="../Data/6AII-1Ks-16x/Results/Grain-Images_high_strain/070/"        #select folder

for grain in grainCopy:
    currentGrain=DicMap.grainList[grain-1]
    strain="%.3f" % myGrainListDic[grain-1,2]
    name="Grain_"+str(grain)+"__"+strain+".png"
    path=folder+name
    currentGrain.calcSlipTraces()
    currentGrain.plotMaxShear(plotSlipTraces=True, plotShearBands=True, vmax=5)
    plt.savefig(path)
    print(grain)

    name="Grain_"+str(grain)+"__"+strain+"__no-bands.png"
    path=folder+name
    currentGrain.plotMaxShear(plotSlipTraces=False, plotShearBands=False, vmax=5)
    plt.savefig(path)
    


In [ ]:
### Large grains

In [ ]:
myGrains_copy=np.copy(myGrainListDic_size)
grainSorted=myGrains_copy[-10:,0]
grainSorted=list(map(int, grainSorted))
print(grainSorted)

In [ ]:
#### Copy array manualy and select

In [ ]:
grainCopy=(59, 57, 18, 159, 129, 70, 133, 121)
folder="../Data/6AII-1Ks-16x/Results/Grain-Images_large/070/"        #select folder

for grain in grainCopy:
    currentGrain=DicMap.grainList[grain-1]
    strain="%.3f" % myGrainListDic[grain-1,2]
    name="Grain_"+str(grain)+"__"+strain+".png"
    path=folder+name
    currentGrain.calcSlipTraces()
    currentGrain.plotMaxShear(plotSlipTraces=True, plotShearBands=True, vmax=5)
    plt.savefig(path)
    print(grain)

    name="Grain_"+str(grain)+"__"+strain+"__no-bands.png"
    path=folder+name
    currentGrain.plotMaxShear(plotSlipTraces=False, plotShearBands=False, vmax=5)
    plt.savefig(path)
    


In [ ]:
plt.text?

In [ ]:
class ab_Maps(object):
    def __init__(self):
        self.f11 = None
        self.f12 = None
        self.f22 = None
        self.Max_shear = None
        
    def creat_f11(self, def_map):
        self.f11 = Map_transform()
        self.f11.transform(def_map)
        
    def creat_def_map(self):
        self.max_shear = Map_transform()
        self.max_shear.transform(max_shear)
        
        self.f11 = Map_transform()
        self.f11.transform(f11_map)
        
        self.f12 = Map_transform()
        self.f12.transform(f12_map)
        
        self.f22 = Map_transform()
        self.f22.transform(f22_map)
        
          
class  Map_transform():
    def __init__(self):
        self.alpha = None
        self.beta = None
        self.mixed = None
        
    def transform(self, def_map):
        cur_map = np.zeros(np.shape(def_map))
        
        self.alpha = np.zeros(np.shape(def_map))
        self.beta = np.zeros(np.shape(def_map)) 
        self.mixed = np.zeros(np.shape(def_map))
        
        colum=0
        
        for j in def_map:
            row=0
            for i in j:
                if DicMap.grains[colum,row] in a_grains:
                    self.alpha[colum,row] = i
                    self.beta[colum,row] = -1
                    self.mixed = -1

                elif DicMap.grains[colum,row] in b_grains:
                    self.beta[colum,row] = i
                    a_maxshear[colum,row] = -1
                    self.mixed = -1

                else:
                    self.alpha[colum,row] = -1
                    self.beta[colum,row] = -1
                    mixed_maxshear[colum,row] = i

                row += 1
            colum += 1    
    


In [ ]:
def start_ab_seperation(self_map, self_boundaries ,xMin, xMax, yMin, yMax, a_value = 85, b_value = 10):
    global cropped_mask
    global cropped_boundary
    global cropped_boundary_1
    global max_shear
    global f11_map
    global f12_map
    global f22_map
    x_upper=DicMap.xDim + xMin
    y_upper=DicMap.yDim + yMin
    cropped_mask=self_map[yMin:y_upper,xMin:x_upper]
    cropped_boundary=self_boundaries[yMin:y_upper,xMin:x_upper]
    cropped_boundary_1=np.zeros((y_upper - yMin, x_upper - xMin))
    
    max_shear = DicMap.max_shear[yMin: -yMax, xMin: -xMax]
    f11_map = DicMap.f11[yMin: -yMax, xMin: -xMax]
    f12_map = DicMap.f12[yMin: -yMax, xMin: -xMax]
    f22_map = DicMap.f22[yMin: -yMax, xMin: -xMax]

    
    
    row=0
    for i in cropped_boundary:
        colum=0
        for j in i:
            if j == 255:
                cropped_boundary_1[row,colum] = 1
            elif j == 0:
                cropped_boundary_1[row,colum] = 0
            else:
                print("error")
            colum += 1
        row += 1

### start of "ab_fraction code

    row=0
    a=0
    global grains
    global a_grains
    global b_grains
    global mixed_grains
    global ab_stat
    ab_stat=["Alpha", 0,"Beta", 0,"Mixed", 0]
    grains= dict()
    a_grains= dict()
    b_grains= dict()
    mixed_grains= dict()
    grains_temp= dict()
    area_map=np.shape(DicMap.grains)[0]*np.shape(DicMap.grains)[1]

    
    ### This loop asign an average greyscale value to each grain, based on the ab_mask
    for x in DicMap.grains:
        colum=0
        for y in x:
            if y not in grains_temp:
                grains_temp[y] = [1,0,0]
                grains_temp[y][1] = cropped_mask[row, colum]/255
            else:
                grains_temp[y][0] += 1
                grains_temp[y][1] += cropped_mask[row, colum]/255
            colum += 1
        row +=1
        
    for z in grains_temp:
        u = grains_temp[z][1] / grains_temp [z][0] * 100
        grains[z] = int(u)
        if u > a_value:
            ab_stat[1] += 1
            a_grains[z] = 1
        elif u < b_value:
            ab_stat[3] += 1
            b_grains[5] = 1
        else:
            ab_stat[5] += 1
            mixed_grains[z] = 1
            
    ab_stat[1] = int(ab_stat[1]/len(grains)*100)
    ab_stat[3] = int(ab_stat[3]/len(grains)*100)
    ab_stat[5] = int(ab_stat[5]/len(grains)*100)
            
    


def print_maps(self_map):   
        fig= plt.figure(figsize=(10,7))
        plt.imshow(self_map,cmap='viridis',vmin=0,vmax=0.05);
        plt.tick_params(axis='both', which='both', bottom='off', top='off', labelbottom='off', right='off', left='off', labelleft='off')
        cbar = plt.colorbar(pad=0.01)
        cbar.set_label(label="$\gamma_{eff}$",size=40)
        cbar.ax.tick_params(labelsize=20)

In [ ]:
vbf=xyz()

In [ ]:
value = 7

In [ ]:
class fgh():
    def __init__(self, value):
        self.five = value

In [ ]:
ytr=fgh(value)